# ML DEPLOYMENT WITH GRADIO - {"BIGMART SALES" DATASET}

## 1. Importing Modules

In [1]:
import numpy as np
import pandas as pd

from pickle import dump, load

import gradio as gr

import warnings
warnings.filterwarnings('ignore')

## 2. Loading Data and Model

### 2.1 Loading Xtrain and Model

In [2]:
def ft_exp(x):
    y = x**(1/1.2)
    return y

def ft_sqrt(x):
    y = x**(1/2)  
    return y

In [3]:
X = load(open('bms_X_prod.pkl','rb'))
pipe = load(open('bms_mdl_prod.pkl','rb'))

### 2.2 Loading Test Data

In [4]:
te = pd.read_pickle('bms_FE_test_final.pkl')

print(f'Shape of the test dataset : {te.shape}')

Shape of the test dataset : (100, 11)


In [5]:
te.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Category,Outlet_Age
2412,13.000,Non Edible,0.041817,Household,254.5014,Small,Tier 2,Supermarket Type1,4845.0266,Non Consumables,9
4984,10.195,Low Fat,0.126383,Canned,112.1886,Small,Tier 1,Supermarket Type1,667.1316,Foods,16
953,17.750,Regular,0.112986,Baking Goods,185.4240,Small,Tier 3,Grocery Store,745.6960,Foods,15
5100,8.970,Non Edible,0.093159,Household,55.7956,Medium,Tier 1,Supermarket Type1,1037.3164,Non Consumables,14
4297,14.500,Regular,0.069042,Canned,41.4454,Small,Tier 3,Grocery Store,41.9454,Foods,15


### 2.3 Preparing App Samples from Test Data

In [6]:
Xte = te.drop(columns='Item_Outlet_Sales')
yte = te['Item_Outlet_Sales']

samp = Xte.sample(5).index.values.tolist()
samp

print('App Samples with Labels ...')
te_df = pd.concat([Xte.loc[samp],yte.loc[samp]],axis=1).head(10)

App Samples with Labels ...


In [7]:
print('App Samples without Labels ...')
gr_samples = Xte.loc[samp].values.tolist()

App Samples without Labels ...


## 3. Gradio App Development

### 3.1 Gradio App Prediction Function

In [8]:
def bms_prediction(Item_Weight, Item_Fat_Content, Item_Visibility, Item_Type, Item_MRP, Outlet_Size, Outlet_Location_Type, Outlet_Type, Item_Category, Outlet_Age):
    
    Item_Weight = float(Item_Weight)
    Item_Fat_Content = str(Item_Fat_Content)
    Item_Visibility = float(Item_Visibility)
    Item_Type = str(Item_Type)
    Item_MRP = float(Item_MRP)
    Outlet_Size = str(Outlet_Size)
    Outlet_Location_Type = str(Outlet_Location_Type)
    Outlet_Type = str(Outlet_Type)
    Item_Category = str(Item_Category)
    Outlet_Age = int(Outlet_Age)    
    
    data = np.array([Item_Weight, Item_Fat_Content, Item_Visibility, Item_Type, Item_MRP, Outlet_Size, Outlet_Location_Type, Outlet_Type, Item_Category, Outlet_Age],dtype=object).reshape(1,10)
    # print(data)
    res = pipe.predict(data)
    # print(res)
    
    return f'Predicted Sales : {round(res[0],4)}'

In [9]:
res = bms_prediction(5.985, 'Low Fat', 0.096337184, 'Frozen Foods', 125.2678, 'Small', 'Tier 2', 'Supermarket Type1', 'Foods', 6)
print(res)

Predicted Sales : 1895.8969


### 3.2 Gradio App Interface for Input and Output

In [10]:
iface = gr.Interface(bms_prediction, 
        inputs=[
                gr.Textbox(default=None, label='Item Weight', info='Weight of the item'),                     # item_weight
                gr.Dropdown(X[X.columns.tolist()[1]].unique().tolist(), label='Item Fat Content', info='Fat content type of the item'),
                gr.Textbox(default=None, label='Item Visibility', info='Visibility value of the item'),       # item_vis
                gr.Dropdown(X[X.columns.tolist()[3]].unique().tolist(), label='Item Type', info='Type of the item'),
                gr.Textbox(default=None, label='Item MRP', info='MRP value of the item'),                          # item_mrp
                gr.Dropdown(X[X.columns.tolist()[5]].unique().tolist(), label='Outlet Size', info='Size of the outlet'),
                gr.Dropdown(X[X.columns.tolist()[6]].unique().tolist(), label='Outlet Location Type', info='Location type of the outlet'),
                gr.Dropdown(X[X.columns.tolist()[7]].unique().tolist(), label='Outlet Type', info='Type of the outlet'),
                gr.Dropdown(X[X.columns.tolist()[8]].unique().tolist(), label='Item Category', info='Category of the item'),
                gr.Slider(X[X.columns[9]].min(),X[X.columns[9]].max(),4, step=1, label='Outlet Age', info='Age of the outlet')   
               ], 
        outputs="text",
        layout='horizontal', theme='soft', 
        title='BigMart Sales Predictor App', 
        description='Provide Details of the Item and Outlet', 
        article='Created by: Manish Sharma',
        examples=gr_samples) 

### 3.3 Gradio App Launch

In [14]:
print(gr_samples[:1])

[[11.65, 'Regular', 0.058719726, 'Baking Goods', 171.1422, 'Small', 'Tier 2', 'Supermarket Type1', 'Foods', 9]]


In [15]:
iface.launch(share=False, debug=True, width=1125, height=750)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
